# 🏭 Session 5 Lab: The GenAI File Factory
**Objective:** Move beyond "Chat" and use LLMs to generate structured artifacts (Docs, Code, Data, Slides).

**Prerequisites:**
1. **Ollama** running locally (`ollama serve`)
2. Model pulled: `ollama pull llama3.2`
3. **Pandoc** installed on your OS (`sudo apt install pandoc` or Windows installer)

In [11]:
import ollama
import re
import os
import pandas as pd

# Check if Pandoc is installed
print("Checking Pandoc version:")
!pandoc --version | head -n 1

MODEL="llama3.2:latest"
#MODEL="gemma3:1b"
#MODEL="gemma3:270m"

Checking Pandoc version:
pandoc 3.8.2.1


In [12]:
# --- HELPER FUNCTIONS ---

def query_llm(prompt, model=MODEL):
    """Sends a prompt to the local Ollama instance."""
    print(f"🤖 Asking {model}...")
    response = ollama.chat(model=model, messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

def extract_code_block(text, language=None):
    """
    Extracts content inside ```markdown or ```python blocks.
    Handles variations like '```markdown ' (trailing space) or different newlines.
    """
    # Pattern explanation:
    # ```         -> Start of block
    # [^\n]* -> Matches language name and any extra spaces until end of line
    # \n          -> Matches the newline after the language name
    # (.*?)       -> Captures the actual content (non-greedy)
    # \n?         -> Matches optional newline before closing
    # ```         -> End of block
    pattern = r"```[^\n]*\n(.*?)\n?```"

    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[0].strip()

    # Fallback: If no blocks found, try to strip just the markdown quotes if present
    # This handles cases where the LLM might output ```markdown content ``` (one line)
    return text.replace("```markdown", "").replace("```python", "").replace("```", "").strip()

--- 
## 1. Document Generation (Markdown -> DOCX / PDF)
We will ask Llama 3.2 to write a structured **Project Charter** in Markdown, then convert it.

In [13]:
prompt_doc = """
You are a Senior Project Manager.
Write a Project Charter for a new AI Mobile App called 'PlantDoctor'.
Use Markdown formatting.
Include these Headers (H1, H2):
- Project Title
- Executive Summary
- Objectives (Bullet points)
- Key Stakeholders (Table)
- Risks

Output ONLY the markdown code block.
"""

# 1. Generate
raw_doc = query_llm(prompt_doc)
clean_doc = extract_code_block(raw_doc)

# 2. Save to .md
with open("project_charter.md", "w", encoding="utf-8") as f:
    f.write(clean_doc)

print("✅ Markdown file saved: project_charter.md")
print(clean_doc[:200] + "...[truncated]")

🤖 Asking llama3.2:latest...
✅ Markdown file saved: project_charter.md
# Project Charter for PlantDoctor AI Mobile App

## Project Title
### PlantDoctor: Revolutionizing Urban Gardening with AI-Powered Insights


## Executive Summary

PlantDoctor is a cutting-edge AI mob...[truncated]


In [14]:
# 3. Convert using Pandoc
# Convert to Word
!pandoc project_charter.md -o project_charter.docx
print("📄 Converted to DOCX")

# Convert to PDF (Requires LaTeX, fallback to HTML if not installed)
# !pandoc project_charter.md -o project_charter.pdf
# OR convert to HTML first for easy viewing
!pandoc project_charter.md -o project_charter.html
print("🌐 Converted to HTML (for preview)")

📄 Converted to DOCX
🌐 Converted to HTML (for preview)


### 1.1 Using a Word Template (Reference Doc)
To make the document look professional (corporate colors/fonts), we use a `--reference-doc`.
*Note: Ensure you have a 'template.docx' in the folder. If not, this step uses default styling.*

In [15]:
# Check if user has a template, otherwise warn
if os.path.exists("../../../data/timbrado.docx"):
    !pandoc project_charter.md --reference-doc="../../../data/timbrado.docx" -o project_charter_styled.docx
    print("🎨 Created STYLED Word document using template.docx")
else:
    print("⚠️ No 'template.docx' found. Please create a blank Word doc with your custom styles and name it 'template.docx' to test this feature.")

🎨 Created STYLED Word document using template.docx


--- 
## 2. Web Page Generation (HTML + CSS + JS)
We will ask the LLM to generate a single-file interactive dashboard.

In [16]:
prompt_web = """
Create a single HTML file that includes embedded CSS and JavaScript.
The page should contain:
1. A modern, dark-mode design.
2. A live digital clock in the center that updates every second.
3. A simple image slideshow (use placeholders like 'https://placehold.co/600x400') below the clock.
4. A 'Toggle Theme' button to switch between dark and light mode.

Output ONLY the raw HTML code block.
"""

# 1. Generate
raw_web = query_llm(prompt_web)
clean_web = extract_code_block(raw_web, language="html")

# 2. Save
with open("dashboard.html", "w", encoding="utf-8") as f:
    f.write(clean_web)

print("🌍 Web page saved: dashboard.html. Open this file in your browser to see the Clock and Slideshow!")

🤖 Asking llama3.2:latest...
🌍 Web page saved: dashboard.html. Open this file in your browser to see the Clock and Slideshow!


--- 
## 3. Code Generation & Execution (Python)
We will generate Python code to perform a mathematical simulation (Approximating Pi).

In [17]:
prompt_code = """
Write a complete Python script to estimate the value of Pi using the Monte Carlo method.
- Use 100,000 iterations.
- Print the estimated value vs the actual value from the math library.
- Do not use any input() functions (it needs to run unattended).
Output ONLY the python code block.
"""

# 1. Generate
raw_code = query_llm(prompt_code)
clean_code = extract_code_block(raw_code, language="python")

# 2. Save
with open("calc_pi.py", "w", encoding="utf-8") as f:
    f.write(clean_code)

print("🐍 Python script saved. Running it now...\n")

# 3. Execute the generated code
!python calc_pi.py

🤖 Asking llama3.2:latest...
🐍 Python script saved. Running it now...

Estimated Pi: 3.13384
Actual Pi: 3.141592653589793
Time taken: 0.03189969062805176 seconds


--- 
## 4. Presentation Slides (Markdown -> PPTX)
Pandoc can turn Markdown headings into PowerPoint slides.
- `H1` (#) = Title Slide
- `H2` (##) = New Slide

In [20]:
prompt_slides = """
Generate a presentation outline about 'The Future of Generative AI' in Markdown.
Structure it for Pandoc conversion:
- Use '#' for the Presentation Title.
- Use '##' for 5 distinct slides.
- Use bullet points for content.

Per slide, suggest one figure that is relevant for the content.
Output ONLY the markdown code.
"""

raw_slides = query_llm(prompt_slides)
clean_slides = extract_code_block(raw_slides)

with open("presentation.md", "w", encoding="utf-8") as f:
    f.write(clean_slides)

# Convert to PPTX
!pandoc presentation.md -o presentation.pptx
print("📽️ Presentation generated: presentation.pptx")

🤖 Asking llama3.2:latest...
📽️ Presentation generated: presentation.pptx


--- 
## 5. Data Engineering (CSV Generation & Excel Export)
We will ask the LLM to generate dummy data, then we use Python to load it into Excel.

In [19]:
prompt_data = """
Generate a CSV file with 50 rows of dummy E-commerce sales data.
Columns: TransactionID, Date, Product, Category, Amount, CustomerCity.
Make sure the data looks realistic.
Output ONLY the CSV data block.
"""

# 1. Generate
raw_csv = query_llm(prompt_data)
clean_csv = extract_code_block(raw_csv)

# 2. Save to CSV
with open("sales_data.csv", "w", encoding="utf-8") as f:
    f.write(clean_csv)
print("💾 CSV Saved.")

# 3. Process with Pandas and Save to Excel
try:
    df = pd.read_csv("sales_data.csv")
    print(f"📊 Loaded Data: {df.shape[0]} rows found.")

    # Simple transformation: Calculate Tax
    if 'Amount' in df.columns:
        df['Tax'] = df['Amount'] * 0.23
        df['Total'] = df['Amount'] + df['Tax']

    # Save to Excel
    df.to_excel("sales_report.xlsx", index=False)
    print("📈 Converted to Excel: sales_report.xlsx")
    display(df.head())
except Exception as e:
    print(f"❌ Error processing CSV: {e}")

🤖 Asking llama3.2:latest...
💾 CSV Saved.
📊 Loaded Data: 50 rows found.
📈 Converted to Excel: sales_report.xlsx


,TransactionID,Date,Product,Category,Amount,CustomerCity,Tax,Total
0,1,2023-01-01,Smartphone,Mobile Phones,129.99,Bangkok,29.8977,159.8877
1,2,2023-01-02,Gaming Laptop,Laptops,499.99,Singapore,114.9977,614.9877
2,3,2023-01-03,E-book,Books,11.99,Hong Kong,2.7577,14.7477
3,4,2023-01-04,Washing Machine,Appliances,299.99,Kuala Lumpur,68.9977,368.9877
4,5,2023-01-05,Smartwatch,Mobile Accessories,79.99,Taipei,18.3977,98.3877
